In [11]:
import pandas as pd
import numpy as np
import collections
from collections import Counter

In [66]:
with open('train.txt', 'r') as f:
    sess_train = f.read().splitlines()
with open('test.txt', 'r') as f:
    sess_test = f.read().splitlines()

In [90]:
sess_train_lp = []
for sess in sess_train:
    look_items, pur_items = sess.split(';')
    look_items = list(map(int, look_items.split(',')))
    if len(pur_items) > 0:
        pur_items = list(map(int, pur_items.split(',')))
    else:
        pur_items = []
    sess_train_lp.append([look_items, pur_items])
    
# Create test array splitted by looks (look_items) and purchases (pur_items)
sess_test_lp = []
for sess in sess_test:
    look_items, pur_items = sess.split(';')
    look_items = list(map(int, look_items.split(',')))
    if len(pur_items) > 0:
        pur_items = list(map(int, pur_items.split(',')))
    else:
        pur_items = []
    sess_test_lp.append([look_items, pur_items])
sess_test_lp

[[[6, 7, 8], []],
 [[13, 14, 15], []],
 [[22, 23], []],
 [[28, 29, 30, 31, 32, 33], []],
 [[40, 41], []],
 [[43, 44, 43, 45, 43, 45, 43, 46], []],
 [[50, 51, 47, 52, 49, 53, 54, 55, 56, 57, 58], []],
 [[63, 68, 69, 70, 66, 61, 59, 61, 66, 68], [66, 63]],
 [[75], []],
 [[79, 80, 81, 82, 83], []],
 [[94,
   95,
   94,
   96,
   97,
   98,
   99,
   100,
   101,
   80,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   103,
   112,
   113],
  []],
 [[125, 126, 127, 128], []],
 [[135, 136], []],
 [[145, 146, 147, 148, 149, 150, 151, 152, 153, 154], []],
 [[158, 159, 160, 159, 161, 162], [162]],
 [[167, 168, 169, 170, 79, 171, 170, 79, 172], []],
 [[177,
   178,
   179,
   180,
   181,
   178,
   182,
   183,
   184,
   185,
   186,
   90,
   187,
   182,
   184,
   182,
   184,
   188,
   189,
   190,
   191],
  []],
 [[93, 193, 194], []],
 [[122, 197, 182], []],
 [[200, 201, 202, 203, 204], [201, 205]],
 [[208, 209, 210, 211, 212, 213, 214, 215], []],
 [[21

In [91]:
sess_train_l = [row[0] for row in sess_train_lp]
sess_train_l_np = np.array( [id_n for sess in sess_train_l for id_n in sess] )

# Array of unique ids and looks in train data
sess_train_l_cnt = np.transpose(np.unique(sess_train_l_np, return_counts=True))
sess_train_l_cnt

array([[     0,      6],
       [     1,      6],
       [     2,      9],
       ...,
       [102804,      1],
       [102805,      1],
       [102806,      1]], dtype=int64)

In [92]:
sess_train_p = [row[1] for row in sess_train_lp]
sess_train_p_np = np.array( [id_n for sess in sess_train_p for id_n in sess] )

# Array of unique ids and purchases in train dataset
sess_train_p_cnt = np.transpose(np.unique(sess_train_p_np, return_counts=True))
sess_train_p_cnt

array([[     5,      1],
       [     6,      2],
       [     7,      2],
       ...,
       [102417,      1],
       [102462,      1],
       [102646,      1]], dtype=int64)

In [94]:
sess_train_l_cnt = sess_train_l_cnt[sess_train_l_cnt[:,1].argsort()][::-1]
sess_train_p_cnt = sess_train_p_cnt[sess_train_p_cnt[:,1].argsort()][::-1]

In [77]:
def prec_rec_metrics(session, reccomendations, k):
    purchase = 0
    for ind in reccomendations:
        if ind in session:
            purchase += 1 
    precision = purchase / k
    recall = purchase / len(session)
    return(precision, recall)

In [95]:
prec_at_1_tr_l, rec_at_1_tr_l = [], []
prec_at_5_tr_l, rec_at_5_tr_l = [], []
k1, k5 = 1, 5
for i, sess_p in enumerate(sess_train_p):
    # skip sessions without purchases
    if sess_p == []: continue
    
    # looks ids
    sess_l = sess_train_l[i]
        # sorted looks ids indices in sess_train_l_cnt array
    # sort in accordance with looks counts
    l_ind_sess = []
    for j in range(len(sess_l)):
        l_ind_sess.append(np.where(sess_train_l_cnt[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(sess_l))
    if num_of_recs_k1 == 0: continue
    recs_k1 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tr_l.append(prec_1)
    rec_at_1_tr_l.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    recs_k5 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tr_l.append(prec_5)
    rec_at_5_tr_l.append(rec_5)

In [96]:
avg_prec_at_1_tr_l = np.mean(prec_at_1_tr_l)
avg_rec_at_1_tr_l = np.mean(rec_at_1_tr_l)
avg_prec_at_5_tr_l = np.mean(prec_at_5_tr_l)
avg_rec_at_5_tr_l = np.mean(rec_at_5_tr_l)

In [97]:
avg_rec_at_5_tr_l

0.8250351974645211